In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly import graph_objects as go

import matplotlib
import plotly
import sklearn
import torch

from torch import nn, optim

from IPython.display import display
plt.style.use("ggplot")

print("package版本信息：")
print("numpy:      ", np.__version__)
print("pandas:     ", pd.__version__)
print("matplotlib: ", matplotlib.__version__)
print("sklearn:    ", sklearn.__version__)
print("seaborn:    ", sns.__version__)
print("plotly:     ", plotly.__version__)
print("PyTorch:     ", torch.__version__)

package版本信息：
numpy:       1.19.2
pandas:      1.0.1
matplotlib:  3.3.1
sklearn:     0.23.2
seaborn:     0.11.1
plotly:      4.14.1
PyTorch:      1.2.0


# Pytorch-Transformer

这里介绍了Pytorch里 transformer 层的实现模块.

In [2]:
from torch import nn

## MultiheadAttention Layer

需要注意的是，pytorch源码中 `MultiheadAttention` 被放在了 Non-linear Activations 这一类中（并且是放在`activation.py`文件中的）。

`MultiheadAttention(embed_dim, num_heads, dropout=0.0, bias=True, add_bias_kv=False, add_zero_attn=False, kdim=None, vdim=None)`
+ 参数
  + `embed_dim`，这个是所有head的总维度，分配到每个head的维度还需要除以下面的 num_heads.
  + `num_heads`，
  + `bias`，
  + `add_bias_kv`
  + `add_zero_attn`
  + `kdim`，自定义 key 中的特征维度
  + `vidm`，自定义 value 中的特征维度


+ `forward(query, key, value, key_padding_mask=None, need_weights=True, attn_mask=None)`
  + `query`, `key`, `value`，self-attention的三个输入向量——这三个输入向量会被用于生成对应的Q,K,V
    + `query.shape` = $(L, N, E)$，
    + `key.shape` = $(S, N, E)$
    + `value.shape` = $(S, N, E)$
  
  其中 $N$ 是batch size，$E$ 是 embedding 维度——**它必须等于参数中的`embed_dim`**，这个embedding 维度，既是输入的，也是输入的——**self-attention层输入和输入的特征维度必须一样**  ；  
  $S$ 是**输入**序列的长度，$L$ 是**输出**序列的长度——**输入和输出序列的长度可以不一样**.
  
  + `key_padding_mask`，
  + `need_weights`，bool，表示是否需要输出attention的weights
  + `attn_mask`，2维或者3维的mask


+ `forward()`方法返回值
  + `attn_output`，shape= $(L, N, E)$，$L$ 由 输入的`query`序列长度决定，$E$ 由输入序列中每个word的特征维度确定
  + `attn_output_weights`，shape=$(N, L, S)$，这个权重应该是每个输入序列中，每个word的attention权重.

In [3]:
# 这个 embed_dim 指定的既是输入序列中，每个word的特征维度，也指定了输入序列中，每个word的特征维度
embed_dim = 512
num_heads = 8
multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)

In [4]:
E, S, L, N = embed_dim, 20, 10, 5
query = torch.rand(L, N, E)
key = torch.rand(S, N, E)
value = torch.rand(S, N, E)

attn_output, attn_output_weights = multihead_attn(query, key, value)

In [5]:
print(attn_output.shape)
print(attn_output_weights.shape)

torch.Size([10, 5, 512])
torch.Size([5, 10, 20])


## Encoder Layer

单层Encoder：`nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=2048, dropout=0.1, activation='relu')`
+ 参数
  + `d_model`，输入序列中，每个word的特征个数——它同时也决定了输出序列里每个word的特征个数
  + `nhead`，multiheadattention中的head个数
  + `dim_feedforward`，这个维度设置的是前馈神经网络的节点个数，前馈神经网络的输出节点数还是 `d_model`
  
  
+ `forward(src, src_mask=None, src_key_padding_mask=None)`方法
  + `src`，输入的sequence, `shape` = $(S, N, E)$
  + `src_mask`，输入sequence的mask
  + `src_key_padding_mask`，每个batch中src的padding
  
  
+ `forward()`方法返回值  
  它返回的是和 `src` shape 相同的 tensor.
  
+ Encoder的内部只有**一层**`MultiheadAttention`：`Encoder.forward()`实际执行时，调用方式为`MultiheadAttention.forward(src, src, src)`——`src`会同时作为 query, key, value 传入 self-attention。

多层Encoder：`TransformerEncoder(encoder_layer, num_layers, norm=None)`
+ 参数：
  + `encoder_layer`，是一个`TransformerEncoderLayer`类的示例
  + `num_layers`，指定堆叠的 Encoder 层数
+ `forward()`方法就和`TransformerEncoderLayer`的一致


In [8]:
d_model = 512
nhead = 8
dim_ffn = 2048
batch_size = 10
source_seq_len = 32

encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8, dim_feedforward=dim_ffn)

src = torch.rand(source_seq_len, batch_size , d_model)
out = encoder_layer(src)

print("src.shape: ", src.shape)
print("out.shape: ", out.shape)

src.shape:  torch.Size([32, 10, 512])
out.shape:  torch.Size([32, 10, 512])


## Decoder Layer

单层Decoder：`TransformerDecoderLayer(d_model, nhead, dim_feedforward=2048, dropout=0.1, activation='relu')`

+ 参数：
  和 Encoder 一致

+ `forward(tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None)`  
  这个方法和 Encoder 不一致，它多了一个 memory.
  + `tgt`，输入的序列，`shape`= $(S, N, E)$，
  + `memory`，来自 encoder 层（通常是最后一层）的输出, `shape`=$(S, N, E)$.
  + `tgt_mask`
  + `memory_mask`
  + `tgt_key_padding_mask`
  + `memory_key_padding_mask`
  

+ `forward()`方法返回值  
  它返回的是和 `tgt` shape 相同的 tensor.

+ `Decoder`内部有两层`MultiheadAttention`：
  + 第一层调用时为`MultiheadAttention.forward(tgt, tgt, tgt)`
  + 第二层调用时为`MultiheadAttention.forward(tgt, memory, memory)`。
  
多层Decoder：`TransformerDecoder(decoder_layer, num_layers, norm=None)`


In [12]:
d_model = 512
nhead = 8
dim_ffn = 2048
batch_size = 10
source_seq_len = 32

decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_ffn)

tgt = torch.rand(source_seq_len, batch_size, d_model)
memory = torch.rand(source_seq_len, batch_size, d_model)

decoder_out = decoder_layer(tgt, memory)

print("tgt.shape: ", tgt.shape)
print("memory.shape: ", memory.shape)
print("decoder_out.shape: ", decoder_out.shape)

tgt.shape:  torch.Size([32, 10, 512])
memory.shape:  torch.Size([32, 10, 512])
decoder_out.shape:  torch.Size([32, 10, 512])


# Huggingface-Transformer-PreTraindedModels

这里介绍huggingface编写的transformer包，它里面基于transformer结构，实现了各种模型，包括BERT, DistilBERT, GPT2 等等.

transformer 包里最重要的三个子模块是：
1. [Configuration](https://huggingface.co/transformers/main_classes/configuration.html)  
基础的类是 `PretrainedConfig`（好像也只有这一个类），用于 **导出/导入** 模型的配置。
2. [Models](https://huggingface.co/transformers/main_classes/model.html)  
基础的类有如下三个，用于构建模型：
  + `PreTrainedModel`，pytorch的模型都继承了这个类，继承了 `torch.nn.Module` 类.
  + `TFPreTrainedModel`，Tensorflow2.0里的模型都继承了这个类，继承了 `tf.keras.Model` 类.
  + `FlaxPreTrainedModel`
3. [Tokenizer](https://huggingface.co/transformers/main_classes/tokenizer.html)  
包含两个基础的类，用于处理文本特征：
  + `PreTrainedTokenizer`，这个比较慢
  + `PreTrainedTokenizerFast`，这个比较快
  
上面这三个基础模块，都有如下两个方法用于**导入/导出**对应的配置：
+ `.from_pretrained()`，导入预训练模型的 config/tokenizer/model
  + `pretrained_model_name_or_path`，指定预训练模型的名称或者本地路径
  + `cache_dir`，指定模型的缓存位置
  + `force_download`，bool，指定是否强制下载模型
  + `local_files_only`，bool，指定是否只使用本地模型
  + `mirror`，指定下载的镜像地址
+ `.save_pretrained()`，导出训练好模型的 config/tokenizer/model

**第一次使用这些语句的时候，如果本地没有对应的模型，会下载这些模型**。
  
在上面的3个基础子模块之上，还提供了如下几个子模块工具：

4. [Pipepline](https://huggingface.co/transformers/main_classes/pipelines.html)  
用于快速使用模型，是对上面三个类的封装
5. [Trainer](https://huggingface.co/transformers/main_classes/trainer.html)  
用于快速训练或者 fine-tune 模型

transformer源码中，所有的模型结构都存放在 `src/transformers/models`文件夹下，每个模型对应于一个文件夹（比如BERT对应的就是`bert`文件夹），每个模型的文件夹内，主要有如下几个`.py`文件，其中的 `xxx` 是对应模型的名称：
+ **`configuration_xxx.py`**，编写了该模型对应的`PretrainedConfig`子类，比如BERT就是`BertConfig`。
+ **`modeling_xxx.py`**，存放了pytorch编写的模型结构，要用到的模型类都放在这里面
+ `modeling_tf_xxx.py`，存放tensorflow编写的模型结构
+ `modeling_flax_xxx.py`
+ **`tokenization_xxx.py`**，用于 分词的 实现类，比如BERT就是`BertTokenizer`类
+ `tokenization_xxx_fast.py`，快速分词的实现类
+ `convert_*.py`，用于将其它配置文件转换成对应的模型。


可用的模型列表见官网 [Pretrained models](https://huggingface.co/transformers/pretrained_models.html).

我一般常用的两个模型是 **BERT** 和 **Distil-BERT**，所以下面会着重关注这两个模型里的实现。

+ [BERT Models](https://huggingface.co/transformers/model_doc/bert.html) 包含如下类（以pytorch为例）
  + `BertConfig`，配置类.  
  位于`configuration_bert.py`文件中，该文件中也只有这一个配置类
  + `BertTokenizer`，实现 `WordPiece` 分词的类.  
  位于`tokenization_bert.py`文件中，该文件中还有 `BasicTokenizer` 和 `WordpieceTokenizer` 两个类，不过`BertTokenizer`会调用它们.
  + `BertModel`，最基本的 BERT 模型类，它返回的是 BERT 模型的原始结果，包括隐藏层状态.
  + `BertForPreTraining`，在 `BertModel`的输出上加了一层处理，以下的几个模型都是如此.
  + `BertForMaskedLM`
  + `BertForNextSentencePrediction`
  + `BertForSequenceClassification`
  + 还有其他的一些基础工具类和其他任务对于的BERT模型类，这里就不介绍了。
  

+ [DistilBERT Models](https://huggingface.co/transformers/model_doc/distilbert.html) 包含如下类

## BERT模型

+ 可用的BERT模型有：
  + bert-base-uncased
  + bert-base-cased
  + bert-large-uncased
  + bert-large-cased
  + bert-base-chinese


+ 可用的Distil-Bert模型有：
  + distilbert-base-uncased
  + distilbert-base-cased

In [20]:
from transformers import BertConfig, BertTokenizer, BertModel

In [26]:
pwd

'D:\\Project-Workspace\\Python-Projects\\DataAnalysis'

In [24]:
cd ..

D:\Project-Workspace\Python-Projects\DataAnalysis


In [39]:
# model_name = "./BERT/bert-pre-trained-models/bert-base-uncased"
model_name = "./BERT/bert-pre-trained-models/distilbert-base-uncased/"

config = BertConfig.from_pretrained(model_name, local_files_only=True)
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path=model_name, local_files_only=True)
model = BertModel.from_pretrained(model_name, local_files_only=True)

print("config.__class__:    ", config.__class__)
print("tokenizer.__class__: ", tokenizer.__class__)
print("model.__class__:     ", model.__class__)

Some weights of the model checkpoint at ./BERT/bert-pre-trained-models/distilbert-base-uncased/ were not used when initializing BertModel: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transfor

config.__class__:     <class 'transformers.models.bert.configuration_bert.BertConfig'>
tokenizer.__class__:  <class 'transformers.models.bert.tokenization_bert.BertTokenizer'>
model.__class__:      <class 'transformers.models.bert.modeling_bert.BertModel'>
